# Reference used: 
- https://dzone.com/articles/algorithm-week-homomorphic

## Sub References: 
- https://www.cs.cmu.edu/~rwh/theses/okasaki.pdf
- https://pdos.csail.mit.edu/papers/otfvec/paper.pdf, 


## Properties of a Homomorphic Function: 

    k.H(m + n + o) = k.H(m) + k.H(n) + k.H(o) = H(k.m) + H(k.n) + H(k.o)

    The homomorphic hash function is:
        H(x) = g^x (mod q)

    here, sum = a + b + c + d + e -> sum is the key and
    a, b, ..., e are the subkeys that are generated from the key

In [64]:
# Reference used: 
## https://dzone.com/articles/algorithm-week-homomorphic
## Sub References: https://www.cs.cmu.edu/~rwh/theses/okasaki.pdf, https://pdos.csail.mit.edu/papers/otfvec/paper.pdf, 

# Properties of a Homomorphic Function: kH(m + n + o) = H(km) + H(kn) + H(ko) = kH(m) + kH(n) + kH(o)
# the homomorphic hash function is H(x) = g^x mod q
# here, sum = a + b + c + d + e -> sum is the key and a, b, ..., e are the subkeys that are generated from the key

# p  [1 2 3 4 5]
# q  [1 2 3 4 5]
# g  [1 2 3 4 5]
# h  [1 2 3 4 5]
# m [[1 2 3 4 5]
#    [6 7 8 9 0]]
# Hs [1 2 3 4 5]
# S [[1 2 3 4 5]
#    [6 7 8 9 0]]
# Hp [1 2 3 4 5]

In [65]:
import random
import math
import pickle

PASS_LEN = 10
BLOCK_LEN = 5

In [66]:
def isPrime(x: int) -> bool:
    for i in range(2, int(math.sqrt(x)) + 1):
        if(x % i == 0): return False
    return True

In [67]:
def keygen(P = 101) -> int:
    keys = []
    i = 1
    # print(f"P: {P}\nQ:")
    while len(keys) < BLOCK_LEN:
        Q = P * i + 1
        if(isPrime(Q) and Q % P == 1):
            # print(f"\t{Q}")
            keys.append(Q)
        i+=1
    k = random.randint(0, BLOCK_LEN-1)
    print(f'keygen({P}): {keys} - {keys[k]}')
    
    return keys[k]

In [68]:
def primegen(start = 101, step = 2) -> list:
    i = start
    primes = []
    while (len(primes) < BLOCK_LEN):
        if(isPrime(i)):
            primes.append(i)
        i+=step

    # print(f'primes({start}, {step}): {primes}')
    return primes

In [69]:
def HomoHash(a: int, b: int, x: int, k = 1) -> int:
    return pow(a, k*x, b)

In [80]:
def obfuscate(x: list, k = [1 for j in range(BLOCK_LEN)]) -> list:
    obfuscated_message = [[0 for j in range(BLOCK_LEN)] for i in range(len(x))] 
    for i in range(len(x)):
        for j in range(BLOCK_LEN):
            obfuscated_message[i][j] = x[i][j]*k[j]
            # print(f"obfuscated_message[{i}][{j}] = x[{i}][{j}]*k[{j}] = {x[i][j]}*{k[j]} = {obfuscated_message[i][j]}")
    return obfuscated_message

# test = [[113, 119, 101, 114, 116], [121, 117, 105, 111, 112]]
# print(obfuscate(test))

In [81]:
def client_hash(x: list, g: list, q: list, p: list, k = [1 for j in range(BLOCK_LEN)])  -> tuple:
    '''
    implements "add": g^k(a+b) (mod q)
    # to compute the hash:
    ## compute the sum of the characters by adding each one
    ## hash that sum by raising g to the power of the sum (mod q)
    '''
    sums = [0 for j in range(BLOCK_LEN)]
    hashed_sums = [0 for j in range(BLOCK_LEN)]
    for i in range(BLOCK_LEN): # for i in range(0, PASS_LEN//BLOCK_LEN + 1): sum+= i%p
        # sum = 0 and replace hashed_sums with sum
        for j in range(len(x)):
            sums[i] += (x[j][i] % p[i])
        sums[i] %= p[i]
        hashed_sums[i] = HomoHash(g[i], q[i], k[i]*(sums[i]))
    return sums, hashed_sums

In [82]:
def server_hash(x: list, g: list, q: list, p: list, k = [1 for j in range(BLOCK_LEN)])  -> tuple:
    '''
    implements "multiply": g^k.a * g^k.b (mod q)
    # hashed_products are the keys computed by each individual server
    # to compute the hash:
    ## compute the hash of each individual character by raising g to the power of the character (mod q)
    ## compute the "sum" of the hashes of the individual characters by multiplying them (mod q)
    '''

    hashed_products = [[1 for j in range(BLOCK_LEN)] for i in range(len(x))]
    products = [0 for j in range(BLOCK_LEN)]
    # for i in range(0, PASS_LEN//BLOCK_LEN + 1): product *= pow(g, q, x[i]%p)
    for i in range(BLOCK_LEN):
        for j in range(len(x)):
            temp = HomoHash(g[i], q[i], k[i]*x[j][i])
            hashed_products[j][i] = temp
            products[i] *= temp
            products[i] %= q[i]
    print(hashed_products)
    return products, hashed_products

In [83]:
# Variables used in the script
# get password from the user
inp = 'qwertyuiop' # input("10 Characters: ") # private
inp += ' '*(PASS_LEN-len(inp))
# hash message
m = [[ord(inp[i+j]) for j in range(BLOCK_LEN)] for i in range(0, PASS_LEN, BLOCK_LEN)] # private
S = [[0 for j in range(BLOCK_LEN)] for i in range(len(m))] # distributed
# Hp = [0 for j in range(BLOCK_LEN)] # public
# Hs = [0 for j in range(BLOCK_LEN)] # public

print(inp)
print(m)

[[113, 119, 101, 114, 116], [121, 117, 105, 111, 112]]


Constants used in the hash function

In [74]:
# Constants used in the hash function
## Choose prime p - 257
p = primegen() # public
print(p)

[101, 103, 107, 109, 113]


In [75]:
## Choose q such that `q % p == 1` or `p | (q - 1)` - 257*6 + 1
q = [keygen(x) for x in p] # public

keygen(101): [607, 809, 1213, 3637, 4243] - 1213
keygen(103): [619, 1031, 1237, 2267, 2473] - 619
keygen(107): [643, 857, 1499, 2141, 3853] - 1499
keygen(109): [1091, 2399, 2617, 3271, 5233] - 2399
keygen(113): [227, 1583, 2713, 2939, 3391] - 2713


In [76]:
## a random number g
g = [random.randint(100, 500) for i in range(BLOCK_LEN)] # public
## generate fuzz factors k
k = [random.randint(15, 100) for i in range(BLOCK_LEN)] # private
# hashes computed at each server, same length as the input + padding
print(f'g: {g}')
print(f'k: {k}')

g: [266, 126, 339, 447, 175]
k: [61, 98, 94, 65, 64]


In [84]:
Hs1 = client_hash(m, g, q, p)
Hp1, S1 = server_hash(m, g, q, p)

Hs = client_hash(m, g, q, p, k)
Hp, S = server_hash(m, g, q, p, k)

[[1115, 527, 93, 883, 1200], [180, 114, 1006, 1985, 2682]]
[[1127, 51, 511, 2325, 1383], [1195, 73, 1211, 1538, 1959]]


In [87]:
print(client_hash(m, g, q, p))
print(server_hash(m, g, q, p))

print(client_hash(m, g, q, p, k))
print(server_hash(m, g, q, p, k))

([32, 30, 99, 7, 2], [658, 35, 1245, 849, 782])
[[1115, 527, 93, 883, 1200], [180, 114, 1006, 1985, 2682]]
([0, 0, 0, 0, 0], [[1115, 527, 93, 883, 1200], [180, 114, 1006, 1985, 2682]])
([32, 30, 99, 7, 2], [878, 9, 584, 2276, 1723])
[[1127, 51, 511, 2325, 1383], [1195, 73, 1211, 1538, 1959]]
([0, 0, 0, 0, 0], [[1127, 51, 511, 2325, 1383], [1195, 73, 1211, 1538, 1959]])


In [86]:
print(f"""\nmessage (primary key):\n{obfuscate(m)}\n
obfuscated sub keys (sent to each server):\n{obfuscate(m, k)}\n
hashes (generated by each server):\n{S}\n
sum_hash (hashing entire message - done by client):\n{Hs1}\n
product (hashing each block and then adding the hashes - done by the distributed network):\n{Hp1}\n
k_sum (obfuscated sum_hash):\n{Hs}\n
k_product (obfuscated product_hash):\n{Hp}\n
k_server (obfuscated hashes calculated on the server):\n{S1}""")


message (primary key):
[[113, 119, 101, 114, 116], [121, 117, 105, 111, 112]]

obfuscated sub keys (sent to each server):
[[6893, 11662, 9494, 7410, 7424], [7381, 11466, 9870, 7215, 7168]]

hashes (generated by each server):
[[1127, 51, 511, 2325, 1383], [1195, 73, 1211, 1538, 1959]]

sum_hash (hashing entire message - done by client):
([32, 30, 99, 7, 2], [658, 35, 1245, 849, 782])

product (hashing each block and then adding the hashes - done by the distributed network):
[0, 0, 0, 0, 0]

k_sum (obfuscated sum_hash):
([32, 30, 99, 7, 2], [878, 9, 584, 2276, 1723])

k_product (obfuscated product_hash):
[0, 0, 0, 0, 0]

k_server (obfuscated hashes calculated on the server):
[[1115, 527, 93, 883, 1200], [180, 114, 1006, 1985, 2682]]


In [79]:

'''
if(sum_hash == product_hash):
    print("Hashes are equal!")

if(k_sum == k_product):
    print("Obfuscated hashes are equal!")
'''

'\nif(sum_hash == product_hash):\n    print("Hashes are equal!")\n\nif(k_sum == k_product):\n    print("Obfuscated hashes are equal!")\n'